In [1]:
import pandas as pd

## Load in the "rosetta stone" file

I made this file using QGIS, the open-source mapping software. I loaded in the US Census 2010 block-level shapefile for DC and the DC police boundaries shapefile [from here](https://opendata.dc.gov/datasets/police-service-areas?geometry=-77.997%2C38.707%2C-76.032%2C39.081). I then used the block centroids, provided by the census, to colect them within each zone. Since the centroids, by nature, are a "half a block" from the nearest street, this is more reliable than a polygon-in-polygon calculation.

I'll write up my steps for that soonest.

In [33]:
rosetta_df = pd.read_csv('../data/oakland/blocks_districts_rosetta.csv')

In [34]:
rosetta_df

,GEOID10,name
0,60019820001006,CCD3
1,60014026001042,CCD3
2,60014026001017,CCD3
3,60014026001024,CCD3
4,60014027001017,CCD3
...,...,...
8410,60014235001001,NaN
8411,60014239022015,NaN
8412,60014228002002,NaN
8413,60014236023007,NaN


In [4]:
# # quick fix to allow sanity merge in QGIS
# rosetta_df["GEOID10_STR"] = "0" + rosetta_df["GEOID10"].astype(str)

In [6]:
# rosetta_df.to_csv('../data/oakland/blocks_districts_rosetta.csv', index=False)

## Load in the population data

I downloaded the population files from [census.data.gov](https://census.data.gov).

In [35]:
# census P3 by block
df_p3 = pd.read_csv('/Volumes/JK_Smarts_Data/census/DECENNIALSF12010.P3_2020-08-21T214208/DECENNIALSF12010.P3_data_with_overlays_2020-08-21T213918.csv')

In [36]:
df_p3.reset_index(inplace=True)

In [37]:
df_p3.drop(0, inplace=True) #drops second column headers

In [73]:
# census P5 by block
df_p5 = pd.read_csv('/Volumes/JK_Smarts_Data/census/DECENNIALSF12010.P5_2020-08-21T230802/DECENNIALSF12010.P5_data_with_overlays_2020-08-21T230757.csv')

In [74]:
df_p5.reset_index(inplace=True)
df_p5.drop(0, inplace=True)

In [76]:
df_p3.shape, df_p5.shape

((23956, 12), (23956, 20))

In [77]:
main_df = df_p3.merge(df_p5, on='GEO_ID')


In [78]:
main_df.shape

(23956, 31)

In [79]:
main_df

,index_x,GEO_ID,NAME_x,P003001,P003002,P003003,P003004,P003005,P003006,P003007,...,P005008,P005009,P005010,P005011,P005012,P005013,P005014,P005015,P005016,P005017
0,1,1000000US060014001001055,"Block 1055, Block Group 1, Census Tract 4001, ...",13,9,1,0,2,0,1,...,1,0,0,0,0,0,0,0,0,0
1,2,1000000US060014001001056,"Block 1056, Block Group 1, Census Tract 4001, ...",85,56,8,0,19,0,0,...,0,2,0,0,0,0,0,0,0,0
2,3,1000000US060014001001058,"Block 1058, Block Group 1, Census Tract 4001, ...",118,85,7,1,21,0,4,...,4,0,4,2,1,1,0,0,0,0
3,4,1000000US060014001001059,"Block 1059, Block Group 1, Census Tract 4001, ...",37,26,1,0,10,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1000000US060014001001064,"Block 1064, Block Group 1, Census Tract 4001, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23951,23952,1000000US060019900000007,"Block 0007, Block Group 0, Census Tract 9900, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23952,23953,1000000US060019900000008,"Block 0008, Block Group 0, Census Tract 9900, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23953,23954,1000000US060019900000009,"Block 0009, Block Group 0, Census Tract 9900, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23954,23955,1000000US060019900000010,"Block 0010, Block Group 0, Census Tract 9900, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
main_df.columns

Index(['index_x', 'GEO_ID', 'NAME_x', 'P003001', 'P003002', 'P003003',
       'P003004', 'P003005', 'P003006', 'P003007', 'P003008', 'GEOID10',
       'index_y', 'NAME_y', 'P005001', 'P005002', 'P005003', 'P005004',
       'P005005', 'P005006', 'P005007', 'P005008', 'P005009', 'P005010',
       'P005011', 'P005012', 'P005013', 'P005014', 'P005015', 'P005016',
       'P005017'],
      dtype='object')

In [81]:
main_df['GEOID10'] = main_df['GEO_ID'].str[9:].astype(int)

In [82]:
main_df.drop(columns=['index_x'], inplace = True)

In [83]:
main_df.drop(columns=['index_y', 'NAME_y'], inplace = True)

In [84]:
main_df.columns

Index(['GEO_ID', 'NAME_x', 'P003001', 'P003002', 'P003003', 'P003004',
       'P003005', 'P003006', 'P003007', 'P003008', 'GEOID10', 'P005001',
       'P005002', 'P005003', 'P005004', 'P005005', 'P005006', 'P005007',
       'P005008', 'P005009', 'P005010', 'P005011', 'P005012', 'P005013',
       'P005014', 'P005015', 'P005016', 'P005017'],
      dtype='object')

In [85]:
rosetta_df.shape

(8415, 2)

In [86]:
rosetta_df.dtypes

GEOID10     int64
name       object
dtype: object

In [87]:
main_df.dtypes

GEO_ID     object
NAME_x     object
P003001     int64
P003002     int64
P003003     int64
P003004     int64
P003005     int64
P003006     int64
P003007     int64
P003008     int64
GEOID10     int64
P005001    object
P005002    object
P005003    object
P005004    object
P005005    object
P005006    object
P005007    object
P005008    object
P005009    object
P005010    object
P005011    object
P005012    object
P005013    object
P005014    object
P005015    object
P005016    object
P005017    object
dtype: object

In [52]:
# need to make all those columns numeric
main_df[['P003001', 'P003002', 'P003003', 'P003004',
       'P003005', 'P003006', 'P003007', 'P003008']] = main_df[['P003001', 'P003002', 'P003003', 'P003004',
       'P003005', 'P003006', 'P003007', 'P003008']].apply(pd.to_numeric)

In [88]:
main_df[['P003001', 'P003002', 'P003003', 'P003004',
       'P003005', 'P003006', 'P003007', 'P003008', 'P005001', 'P005002',
       'P005003', 'P005004', 'P005005', 'P005006', 'P005007', 'P005008',
       'P005009', 'P005010', 'P005011', 'P005012', 'P005013', 'P005014',
       'P005015', 'P005016', 'P005017']] = main_df[['P003001', 'P003002', 'P003003', 'P003004',
       'P003005', 'P003006', 'P003007', 'P003008', 'P005001', 'P005002',
       'P005003', 'P005004', 'P005005', 'P005006', 'P005007', 'P005008',
       'P005009', 'P005010', 'P005011', 'P005012', 'P005013', 'P005014',
       'P005015', 'P005016', 'P005017']].apply(pd.to_numeric)

In [89]:
## Add districts to every block's demo data
block_data = main_df.merge(rosetta_df, on="GEOID10", how="left")

In [90]:
block_data.shape

(23956, 29)

In [91]:
block_data

,GEO_ID,NAME_x,P003001,P003002,P003003,P003004,P003005,P003006,P003007,P003008,...,P005009,P005010,P005011,P005012,P005013,P005014,P005015,P005016,P005017,name
0,1000000US060014001001055,"Block 1055, Block Group 1, Census Tract 4001, ...",13,9,1,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,CCD1
1,1000000US060014001001056,"Block 1056, Block Group 1, Census Tract 4001, ...",85,56,8,0,19,0,0,2,...,2,0,0,0,0,0,0,0,0,CCD1
2,1000000US060014001001058,"Block 1058, Block Group 1, Census Tract 4001, ...",118,85,7,1,21,0,4,0,...,0,4,2,1,1,0,0,0,0,CCD1
3,1000000US060014001001059,"Block 1059, Block Group 1, Census Tract 4001, ...",37,26,1,0,10,0,0,0,...,0,0,0,0,0,0,0,0,0,CCD1
4,1000000US060014001001064,"Block 1064, Block Group 1, Census Tract 4001, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CCD1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23951,1000000US060019900000007,"Block 0007, Block Group 0, Census Tract 9900, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
23952,1000000US060019900000008,"Block 0008, Block Group 0, Census Tract 9900, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
23953,1000000US060019900000009,"Block 0009, Block Group 0, Census Tract 9900, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
23954,1000000US060019900000010,"Block 0010, Block Group 0, Census Tract 9900, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [92]:
# drop rows where "name" (the city council title) is NaN
block_data.dropna(subset=['name'], inplace=True)

In [93]:
block_data.shape

(6284, 29)

In [94]:
block_data.to_csv('./temp_data/oakland_councildistricts_population.csv', index=False)

In [95]:
## Check for duplicates
block_data.duplicated(subset='GEOID10', keep='first').sum()

0

In [96]:
import numpy as np
pivot = pd.pivot_table(block_data, index="name", aggfunc=np.sum)

In [97]:
pivot

,GEOID10,P003001,P003002,P003003,P003004,P003005,P003006,P003007,P003008,P005001,...,P005008,P005009,P005010,P005011,P005012,P005013,P005014,P005015,P005016,P005017
name,,,,,,,,,,,,,,,,,,,,,
CCD1,62594620351163296,57447,33179,12728,284,5702,96,1910,3548,57447,...,266,2787,5356,2432,282,145,84,8,1644,761
CCD2,50171747073804280,57094,17386,9383,301,22458,162,4510,2894,57094,...,222,2013,8419,2667,295,153,119,16,4288,881
CCD3,74177935741245184,57196,17378,22047,408,9845,142,3868,3508,57196,...,207,2596,7688,2382,425,194,99,15,3661,912
CCD4,41169646932407104,54691,26119,10004,377,10714,285,3732,3460,54691,...,196,2495,8276,3174,303,161,131,6,3536,965
CCD5,44470420647708944,54681,16335,8863,857,10737,276,14827,2786,54681,...,110,1213,26015,8573,427,608,98,19,14717,1573
CCD6,40089407618662808,54547,12568,23704,380,3925,549,10282,3139,54547,...,108,1757,18372,5831,599,261,81,44,10174,1382
CCD7,64455141851318008,55284,12062,22789,433,2449,712,14291,2548,55284,...,101,1220,25025,8592,506,304,72,33,14190,1328


In [98]:
pivot.reset_index(inplace=True)

In [101]:
# drop columns whose sums make no sense
pivot.drop(columns=['GEOID10'], inplace=True)

In [102]:
pivot.columns

Index(['name', 'P003001', 'P003002', 'P003003', 'P003004', 'P003005',
       'P003006', 'P003007', 'P003008', 'P005001', 'P005002', 'P005003',
       'P005004', 'P005005', 'P005006', 'P005007', 'P005008', 'P005009',
       'P005010', 'P005011', 'P005012', 'P005013', 'P005014', 'P005015',
       'P005016', 'P005017'],
      dtype='object')

In [103]:
pivot

,name,P003001,P003002,P003003,P003004,P003005,P003006,P003007,P003008,P005001,...,P005008,P005009,P005010,P005011,P005012,P005013,P005014,P005015,P005016,P005017
0,CCD1,57447,33179,12728,284,5702,96,1910,3548,57447,...,266,2787,5356,2432,282,145,84,8,1644,761
1,CCD2,57094,17386,9383,301,22458,162,4510,2894,57094,...,222,2013,8419,2667,295,153,119,16,4288,881
2,CCD3,57196,17378,22047,408,9845,142,3868,3508,57196,...,207,2596,7688,2382,425,194,99,15,3661,912
3,CCD4,54691,26119,10004,377,10714,285,3732,3460,54691,...,196,2495,8276,3174,303,161,131,6,3536,965
4,CCD5,54681,16335,8863,857,10737,276,14827,2786,54681,...,110,1213,26015,8573,427,608,98,19,14717,1573
5,CCD6,54547,12568,23704,380,3925,549,10282,3139,54547,...,108,1757,18372,5831,599,261,81,44,10174,1382
6,CCD7,55284,12062,22789,433,2449,712,14291,2548,55284,...,101,1220,25025,8592,506,304,72,33,14190,1328


In [104]:
### Add percentages
pivot['P003001_pct'] = pivot['P003001'] / pivot['P003001']
pivot['P003002_pct'] = pivot['P003002'] / pivot['P003001']
pivot['P003003_pct'] = pivot['P003003'] / pivot['P003001']
pivot['P003004_pct'] = pivot['P003004'] / pivot['P003001']
pivot['P003005_pct'] = pivot['P003005'] / pivot['P003001']
pivot['P003006_pct'] = pivot['P003006'] / pivot['P003001']
pivot['P003007_pct'] = pivot['P003007'] / pivot['P003001']
pivot['P003008_pct'] = pivot['P003008'] / pivot['P003001']
pivot['P005001_pct'] = pivot['P005001'] / pivot['P003001']
pivot['P005002_pct'] = pivot['P005002'] / pivot['P003001']
pivot['P005003_pct'] = pivot['P005003'] / pivot['P003001']
pivot['P005004_pct'] = pivot['P005004'] / pivot['P003001']
pivot['P005005_pct'] = pivot['P005005'] / pivot['P003001']
pivot['P005006_pct'] = pivot['P005006'] / pivot['P003001']
pivot['P005007_pct'] = pivot['P005007'] / pivot['P003001']
pivot['P005008_pct'] = pivot['P005008'] / pivot['P003001']
pivot['P005009_pct'] = pivot['P005009'] / pivot['P003001']
pivot['P005010_pct'] = pivot['P005010'] / pivot['P003001']
pivot['P005011_pct'] = pivot['P005011'] / pivot['P003001']
pivot['P005012_pct'] = pivot['P005012'] / pivot['P003001']
pivot['P005013_pct'] = pivot['P005013'] / pivot['P003001']
pivot['P005014_pct'] = pivot['P005014'] / pivot['P003001']
pivot['P005015_pct'] = pivot['P005015'] / pivot['P003001']
pivot['P005016_pct'] = pivot['P005016'] / pivot['P003001']
pivot['P005017_pct'] = pivot['P005017'] / pivot['P003001']


In [105]:
pivot

,name,P003001,P003002,P003003,P003004,P003005,P003006,P003007,P003008,P005001,...,P005008_pct,P005009_pct,P005010_pct,P005011_pct,P005012_pct,P005013_pct,P005014_pct,P005015_pct,P005016_pct,P005017_pct
0,CCD1,57447,33179,12728,284,5702,96,1910,3548,57447,...,0.004630,0.048514,0.093234,0.042335,0.004909,0.002524,0.001462,0.000139,0.028618,0.013247
1,CCD2,57094,17386,9383,301,22458,162,4510,2894,57094,...,0.003888,0.035258,0.147459,0.046712,0.005167,0.002680,0.002084,0.000280,0.075104,0.015431
2,CCD3,57196,17378,22047,408,9845,142,3868,3508,57196,...,0.003619,0.045388,0.134415,0.041646,0.007431,0.003392,0.001731,0.000262,0.064008,0.015945
3,CCD4,54691,26119,10004,377,10714,285,3732,3460,54691,...,0.003584,0.045620,0.151323,0.058035,0.005540,0.002944,0.002395,0.000110,0.064654,0.017645
4,CCD5,54681,16335,8863,857,10737,276,14827,2786,54681,...,0.002012,0.022183,0.475759,0.156782,0.007809,0.011119,0.001792,0.000347,0.269143,0.028767
5,CCD6,54547,12568,23704,380,3925,549,10282,3139,54547,...,0.001980,0.032211,0.336810,0.106899,0.010981,0.004785,0.001485,0.000807,0.186518,0.025336
6,CCD7,55284,12062,22789,433,2449,712,14291,2548,55284,...,0.001827,0.022068,0.452663,0.155416,0.009153,0.005499,0.001302,0.000597,0.256675,0.024021


In [106]:
pivot.to_csv('../data/oakland/oakland_city_council_districts.csv', index=False)

In [107]:
pivot['P003001'].sum()

390940

In [115]:
subset = pivot[['name', 'P003002_pct', 'P003003_pct', 'P003004_pct', 'P003005_pct', 'P003006_pct', 'P003007_pct', 'P003008_pct', 'P005010_pct']].copy()

In [116]:
subset

,name,P003002_pct,P003003_pct,P003004_pct,P003005_pct,P003006_pct,P003007_pct,P003008_pct,P005010_pct
0,CCD1,0.577558,0.221561,0.004944,0.099257,0.001671,0.033248,0.061761,0.093234
1,CCD2,0.304515,0.164343,0.005272,0.393351,0.002837,0.078993,0.050688,0.147459
2,CCD3,0.303832,0.385464,0.007133,0.172127,0.002483,0.067627,0.061333,0.134415
3,CCD4,0.477574,0.182919,0.006893,0.195901,0.005211,0.068238,0.063265,0.151323
4,CCD5,0.298733,0.162086,0.015673,0.196357,0.005047,0.271155,0.050950,0.475759
5,CCD6,0.230407,0.434561,0.006966,0.071956,0.010065,0.188498,0.057547,0.336810
6,CCD7,0.218182,0.412217,0.007832,0.044299,0.012879,0.258502,0.046089,0.452663


In [119]:
subset.to_csv('../data/oakland/oakland_councildistricts_population_condensed.csv', index=False)